# OP.GG 플레이어 전적 크롤링
* 특정 소환사(플레이어)의 게임 전적을 크롤링
* https://www.op.gg/summoner/userName={nickname} (https://www.op.gg/summoner/userName=아니어떡하라구요)

### requests로 전적 페이지 받아오기

In [1]:
import requests

headers = {
    'accept-language': 'ko'
}
nickname = '아니어떡하라구요'  # 2325 이상인
url = f'https://www.op.gg/summoner/userName={nickname}'
res = requests.get(url, headers=headers)
res

<Response [200]>

### BeautifulSoup으로 soup 만들기

In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(res.text, 'html.parser')
print(soup.prettify()[:500])

<!DOCTYPE html>
<!--
	  ,ad8888ba,   88888888ba         ,ad8888ba,    ,ad8888ba,
	 d8"'    `"8b  88      "8b       d8"'    `"8b  d8"'    `"8b
	d8'        `8b 88      ,8P      d8'           d8'
	88          88 88aaaaaa8P'      88            88
	88          88 88""""""'        88      88888 88      88888
	Y8,        ,8P 88               Y8,        88 Y8,        88
	 Y8a.    .a8P  88                Y8a.    .a88  Y8a.    .a88
	  `"Y8888Y"'   88     dev@op.gg   `"Y88888P"    `"Y88888P"

		 Copyright 


### 각 게임을 soup.select로 가져오기

In [3]:
games = soup.select('.GameItemWrap > .GameItem')
# print(games[0])

### 게임 결과 받아오기

In [4]:
result = []
for game in games:
    result.append(['패배', '승리'][game['data-game-result'] == 'win'])
    # data-game-result가 win이면 '승리', 아니면 '패배'
result[:5]

['패배', '승리', '패배', '승리', '패배']

### 플레이한 챔피언 이름 가져오기

In [5]:
champ = []
for game in games:
    champ.append(game.find(class_='ChampionName').get_text().strip())
champ[:5]

['오른', '볼리베어', '오른', '볼리베어', '파이크']

### Kill/Death/Assist 가져오기

In [6]:
kda = []
for game in games:
    kda.append('/'.join([i.get_text() for i in game.select('.KDA > .KDA > *')]))
    # .KDA > .KDA 의 요소로 있는 K, D, A 각각을 /로 구분
kda[:5]

['0/6/3', '3/5/3', '1/9/4', '5/6/5', '6/9/5']

### 사용한 아이템 가져오기

In [7]:
item = []
for game in games:
    item.append(', '.join([i['alt'] for i in game.select('.Items > .ItemList > .Item:not(:nth-child(4)) > img')]))
    # 4번째 아이템은 장신구(와드 및 허수아비, 칼리스타의 창 등)이므로 제외
item[:5]

['도란의 방패, 거인의 허리띠, 점화석, 덤불 조끼, 판금 장화',
 '도란의 검, 신성한 파괴자, 루비 수정, 탐식의 망치, 곡괭이, 헤르메스의 발걸음',
 '덤불 조끼, 서리벼림 손아귀, 증오의 사슬, 판금 장화',
 '스테락의 도전, 신성한 파괴자, 판금 장화, 덤불 조끼',
 '드락사르의 황혼검, 징수의 총, 톱날 단검, 명석함의 아이오니아 장화']

### 사용한 룬 가져오기

In [8]:
rune = []
for game in games:
    gameid = game['data-game-id']
    summonerid = game['data-summoner-id']
    # gameid, summonerid를 받아옴
    url = f'https://www.op.gg/summoner/matches/ajax/detail/builds/gameId={gameid}&summonerId={summonerid}&moreLoad=1'
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    # 빌드를 보여주는 ajax를 받아와 soup를 만듦
    rune.append(soup.select_one('.perk-page__item--keystone.perk-page__item--active img')['alt'])
    # keystone(핵심 룬) 중 active(활성화된)인 룬을 가져옴
rune[:5]

['봉인 풀린 주문서', '집중 공격', '봉인 풀린 주문서', '집중 공격', '어둠의 수확']

### DataFrame 만들기

In [9]:
import pandas as pd

df_data = {
    'result': result,
    'champ': champ,
    'kda': kda,
    'item': item,
    'rune': rune
}
df = pd.DataFrame(df_data)
df

,result,champ,kda,item,rune
0,패배,오른,0/6/3,"도란의 방패, 거인의 허리띠, 점화석, 덤불 조끼, 판금 장화",봉인 풀린 주문서
1,승리,볼리베어,3/5/3,"도란의 검, 신성한 파괴자, 루비 수정, 탐식의 망치, 곡괭이, 헤르메스의 발걸음",집중 공격
2,패배,오른,1/9/4,"덤불 조끼, 서리벼림 손아귀, 증오의 사슬, 판금 장화",봉인 풀린 주문서
3,승리,볼리베어,5/6/5,"스테락의 도전, 신성한 파괴자, 판금 장화, 덤불 조끼",집중 공격
4,패배,파이크,6/9/5,"드락사르의 황혼검, 징수의 총, 톱날 단검, 명석함의 아이오니아 장화",어둠의 수확
5,패배,볼리베어,4/8/10,"가시 갑옷, 신성한 파괴자, 헤르메스의 발걸음, 곡괭이, 탐식의 망치",집중 공격
6,승리,오른,2/3/12,"덤불 조끼, 터보 충전 마공학 실험장치, 증오의 사슬, 거인의 허리띠, 판금 장화",봉인 풀린 주문서
7,승리,오른,3/3/14,"도란의 방패, 터보 충전 마공학 실험장치, 증오의 사슬, 가시 갑옷, 얼음 방패, ...",봉인 풀린 주문서
8,패배,아크샨,9/7/3,"도란의 검, 크라켄 학살자, 마법사의 최후, 몰락한 왕의 검, 톱날 단검, 광전사의 군화",집중 공격
9,승리,티모,13/7/6,"도란의 반지, 내셔의 이빨, 균열 생성기, 악마의 포옹, 마법사의 최후, 광전사의 군화",집중 공격


## Riot API를 통해 더 자세한 정보 가져오기

In [10]:
api_key = 'RGAPI-3bb7d59a-d4d3-4096-8f7e-7142ea3c99ae'

* api_key는 24시간마다 만료되므로 https://developer.riotgames.com/ 에서 매일 발급받거나 Product Key를 발급받아 사용해야 함

In [11]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.229 Whale/2.10.123.42 Safari/537.36",
    "Accept-Language": "ko,en;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

### 소환사 정보 가져오기

In [12]:
url = f'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{nickname}'
summoner = requests.get(url, headers=headers).json()
summonerName = summoner['name']
summonerName

'아니어떡하라구요'

### 최근 매치 20개 가져오기

In [13]:
puuid = summoner['puuid']
url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?count=20'
matches = requests.get(url, headers=headers).json()
matches

['KR_5448429896',
 'KR_5448372800',
 'KR_5447315779',
 'KR_5434421745',
 'KR_5430983106',
 'KR_5430678306',
 'KR_5430125962',
 'KR_5430051788',
 'KR_5428974331',
 'KR_5428859656',
 'KR_5428866873',
 'KR_5428942849',
 'KR_5428728773',
 'KR_5428499419',
 'KR_5428543169',
 'KR_5427423152',
 'KR_5427267921',
 'KR_5427247722',
 'KR_5427117386',
 'KR_5427065229']

### 가져온 데이터 확인

In [14]:
url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{matches[0]}'
match = requests.get(url, headers=headers).json()
participants = match['info']['participants']
participants[0]

{'assists': 3,
 'baronKills': 0,
 'bountyLevel': 0,
 'champExperience': 6582,
 'champLevel': 10,
 'championId': 516,
 'championName': 'Ornn',
 'championTransform': 0,
 'consumablesPurchased': 1,
 'damageDealtToBuildings': 0,
 'damageDealtToObjectives': 0,
 'damageDealtToTurrets': 0,
 'damageSelfMitigated': 10217,
 'deaths': 6,
 'detectorWardsPlaced': 0,
 'doubleKills': 0,
 'dragonKills': 0,
 'firstBloodAssist': True,
 'firstBloodKill': False,
 'firstTowerAssist': False,
 'firstTowerKill': False,
 'gameEndedInEarlySurrender': False,
 'gameEndedInSurrender': True,
 'goldEarned': 4197,
 'goldSpent': 3800,
 'individualPosition': 'TOP',
 'inhibitorKills': 0,
 'inhibitorTakedowns': 0,
 'inhibitorsLost': 0,
 'item0': 1054,
 'item1': 1011,
 'item2': 3067,
 'item3': 3076,
 'item4': 3047,
 'item5': 0,
 'item6': 3340,
 'itemsPurchased': 10,
 'killingSprees': 0,
 'kills': 0,
 'lane': 'NONE',
 'largestCriticalStrike': 2,
 'largestKillingSpree': 0,
 'largestMultiKill': 0,
 'longestTimeSpentLiving': 

### 각 매치의 데이터를 받아와 저장

In [15]:
matchlist = []
for matchid in matches:
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{matchid}'
    match = requests.get(url, headers=headers).json()
    matchlist.append(match)

In [16]:
mode = []  # 게임 모드
role = []  # 게임 내 역할(및 라인)
champ = []  # 챔피언 이름
level = []  # 레벨
gold = []  # 얻은 골드
cs = []  # 처치한 미니언
vision = []  # 시야 점수
deal = []  # 챔피언에게 입힌 피해량
win = []  # 승리 여부
for match in matchlist:
    participants = match['info']['participants']
    for p in participants:
        if p['summonerName'] == summonerName:
            mode.append(f"{match['info']['gameMode']} {match['info']['gameType']}")
            position = p['individualPosition']
            if position == 'Invalid':
                position = p['lane']
                if position == 'NONE':
                    position = ''
            role.append(f"{position} {p['role']}".strip())
            champ.append(p['championName'])
            level.append(p['champLevel'])
            gold.append(p['goldEarned'])
            cs.append(p['totalMinionsKilled'])
            vision.append(p['visionScore'])
            deal.append(p['totalDamageDealtToChampions'])
            win.append(p['win'])
            break
win

[False,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False]

### DataFrame 만들기

In [17]:
df_data = {
    'mode': mode,
    'role': role,
    'champ': champ,
    'level': level,
    'gold': gold,
    'cs': cs,
    'vision': vision,
    'deal': deal,
    'win': win
}
df2 = pd.DataFrame(df_data)
df2

,mode,role,champ,level,gold,cs,vision,deal,win
0,CLASSIC MATCHED_GAME,TOP SUPPORT,Ornn,10,4197,81,7,4017,False
1,CLASSIC MATCHED_GAME,TOP SOLO,Volibear,15,8776,136,15,18751,True
2,CLASSIC MATCHED_GAME,TOP SOLO,Ornn,13,7749,124,21,10095,False
3,CLASSIC MATCHED_GAME,TOP SOLO,Volibear,14,8859,105,13,15753,True
4,ARAM MATCHED_GAME,SUPPORT,Pyke,12,8832,5,0,6808,False
5,CLASSIC MATCHED_GAME,TOP SOLO,Volibear,15,9512,121,14,20377,False
6,CLASSIC MATCHED_GAME,TOP SOLO,Ornn,15,8485,110,6,9870,True
7,CLASSIC MATCHED_GAME,TOP SOLO,Ornn,16,10534,135,14,15418,True
8,CLASSIC MATCHED_GAME,MIDDLE SOLO,Akshan,16,12892,184,21,17075,False
9,CLASSIC MATCHED_GAME,TOP NONE,Teemo,18,18128,221,15,36506,True


# 결과물

In [18]:
df

,result,champ,kda,item,rune
0,패배,오른,0/6/3,"도란의 방패, 거인의 허리띠, 점화석, 덤불 조끼, 판금 장화",봉인 풀린 주문서
1,승리,볼리베어,3/5/3,"도란의 검, 신성한 파괴자, 루비 수정, 탐식의 망치, 곡괭이, 헤르메스의 발걸음",집중 공격
2,패배,오른,1/9/4,"덤불 조끼, 서리벼림 손아귀, 증오의 사슬, 판금 장화",봉인 풀린 주문서
3,승리,볼리베어,5/6/5,"스테락의 도전, 신성한 파괴자, 판금 장화, 덤불 조끼",집중 공격
4,패배,파이크,6/9/5,"드락사르의 황혼검, 징수의 총, 톱날 단검, 명석함의 아이오니아 장화",어둠의 수확
5,패배,볼리베어,4/8/10,"가시 갑옷, 신성한 파괴자, 헤르메스의 발걸음, 곡괭이, 탐식의 망치",집중 공격
6,승리,오른,2/3/12,"덤불 조끼, 터보 충전 마공학 실험장치, 증오의 사슬, 거인의 허리띠, 판금 장화",봉인 풀린 주문서
7,승리,오른,3/3/14,"도란의 방패, 터보 충전 마공학 실험장치, 증오의 사슬, 가시 갑옷, 얼음 방패, ...",봉인 풀린 주문서
8,패배,아크샨,9/7/3,"도란의 검, 크라켄 학살자, 마법사의 최후, 몰락한 왕의 검, 톱날 단검, 광전사의 군화",집중 공격
9,승리,티모,13/7/6,"도란의 반지, 내셔의 이빨, 균열 생성기, 악마의 포옹, 마법사의 최후, 광전사의 군화",집중 공격


In [19]:
df2

,mode,role,champ,level,gold,cs,vision,deal,win
0,CLASSIC MATCHED_GAME,TOP SUPPORT,Ornn,10,4197,81,7,4017,False
1,CLASSIC MATCHED_GAME,TOP SOLO,Volibear,15,8776,136,15,18751,True
2,CLASSIC MATCHED_GAME,TOP SOLO,Ornn,13,7749,124,21,10095,False
3,CLASSIC MATCHED_GAME,TOP SOLO,Volibear,14,8859,105,13,15753,True
4,ARAM MATCHED_GAME,SUPPORT,Pyke,12,8832,5,0,6808,False
5,CLASSIC MATCHED_GAME,TOP SOLO,Volibear,15,9512,121,14,20377,False
6,CLASSIC MATCHED_GAME,TOP SOLO,Ornn,15,8485,110,6,9870,True
7,CLASSIC MATCHED_GAME,TOP SOLO,Ornn,16,10534,135,14,15418,True
8,CLASSIC MATCHED_GAME,MIDDLE SOLO,Akshan,16,12892,184,21,17075,False
9,CLASSIC MATCHED_GAME,TOP NONE,Teemo,18,18128,221,15,36506,True
